In [623]:
from shapely.geometry import Polygon, Point, mapping, MultiPoint
from shapely.ops import cascaded_union
import alphashape as als
import folium
from h3 import h3
import geopandas as gpd
import json

p1 = Polygon([
      (
        -72.271843,
        43.0091662
      ),
      (
        -72.2728729,
        42.9503918
      ),
      (
        -72.2230911,
        42.9486327
      ),
      (
        -72.2230911,
        43.007911
      ),
      (
        -72.271843,
        43.0091662
      )
    ])
p2 = Polygon([
      (
        -72.1870422,
        43.0048982
      ),
      (
        -72.2454071,
        42.9933479
      ),
      (
        -72.1935654,
        42.9790325
      ),
      (
        -72.2419739,
        42.9664724
      ),
      (
        -72.1746826,
        42.9586839
      ),
      (
        -72.1870422,
        43.0048982
      )
    ])
print(p1.intersects(p2))

def shapely_coordinates_to_geojson(coordinates):
    polygon_boundary = []
    for coordinate in coordinates[0]:
        polygon_boundary.append([coordinate[0], coordinate[1]])
    geojson = {
        "type": "Polygon",
        "coordinates": [polygon_boundary]
    }
    return geojson

def merge(p1, p2, ratio=0.5, resolution=11):
    # ratio defines, in what degree would points belong to p1
    # so ratio = 0 means every trapped and intersecting point belongs to p2
    # and ratio = 1 means every trapped and intersecting point belongs to p1
    
    # resolution's usage is described while defining 'hexgrids_x'
    p1new, p2new = (p1.difference(p2), p2.difference(p1))
    union = cascaded_union([p1, p2])
    points = list(union.exterior.coords)

    polygons_x = []
    # only exterior coords of union IS the concave hull.
    concave_hull = Polygon(list(union.exterior.coords))
    trapped_patches = convex_poly.difference(union)
    for patch in trapped_patches.geoms:
        if concave_hull.contains(patch):

            patch = patch.buffer(0.001)
            polygon_x = shapely_coordinates_to_geojson([list(patch.exterior.coords)])
            polygons_x.append(polygon_x)
    x = p1.intersection(p2)
    x1 = x
    # buffer is because, polyfill returns hexgrids totally contained within the polygon. thus leaving some gaps
    # between intersection and original polygons
    x = x.buffer(0.001)
    # m.choropleth(geo_data=x)
    for coordinates in mapping(x)["coordinates"]:
        geojson_x = shapely_coordinates_to_geojson(coordinates)
        polygons_x.append(geojson_x)
    
    # resolution here is critical. in this case if given <=11 or 12, the hexagon is really large compared 
    # to size of polygons, thus giving weird outcomes. 

    # it can be mapped to area while making the function

    # for example a big polgon if run on resolution 15 would take a lot of time, because the number of grids will be
    # too many. 
    # at the same time, very smaller resolution would lead to less fine division of area.
    hexgrids_x = []
    for poly in polygons_x:
        hexgrids_x.append(h3.polyfill(poly, resolution))
    
    p1new_buffer = []
    p2new_buffer = []
    for hexgrids in hexgrids_x:
        for cell in hexgrids:
            bound = h3.h3_to_geo_boundary(cell)
            poly = Polygon(bound)
            center = Point(h3.h3_to_geo(cell))
            # if center of grid isn't inside, it might be close to p1 but parts of it lie in p2, so 
            # might result in a MultiPolygon with parts of polygon actually inside p2
            if not concave_hull.contains(center):
                continue
            d1 = center.distance(p1new)
            d2 = center.distance(p2new)
            
            if d2 != 0:
                ratio_local = d1/d2
            else:
                # very small value
                ratio_local = 0
            
            if d1/(d1+d2) > ratio:
                p2new_buffer.append(poly)
                # a grid has parts in p2, but some parts in p1. so while cascading_unioning
                # both p1, p2 has parts of this grid
                p1new = p1new.difference(poly)
            else:
                p1new_buffer.append(poly)
                p2new = p2new.difference(poly)
    #         if ratio > 0.01 and ratio <10:


    #             m.choropleth(geo_data=poly, fill_color='red')

    p1new_buffer.append(p1new)
    p2new_buffer.append(p2new)
    p1new_merged = cascaded_union(p1new_buffer)
    p2new_merged = cascaded_union(p2new_buffer)
    p1p2 = [p1, p2]
    p1p2 = cascaded_union(p1p2)
    p1p2_extended = [p1new_merged, p2new_merged]
    p1p2_extended = cascaded_union(p1p2_extended)
    extra_poly = p1p2_extended.difference(concave_hull)
    p1new = p1new_merged.difference(extra_poly)
    p2new = p2new_merged.difference(extra_poly)
    return p1new, p2new

    

m = folium.Map(location=[42.9933479, -72.2454071],
                   tiles='OpenStreetMap', zoom_start=12)
# m.choropleth(geo_data=p1new)
# m.choropleth(geo_data=p2new)
# m

True


In [624]:
p1new, p2new = merge(p1, p2, ratio=0.1, resolution=11)


In [625]:
m.choropleth(geo_data=p1new, fill_color="red")
m.choropleth(geo_data=p2new, fill_color="yellow")

m